<a href="https://colab.research.google.com/github/shawal-mbalire/ai_lab_hack/blob/main/yolo8n_300E_256b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.9/777.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.5 MB/s eta 0:00:00


In [2]:
import os
import cv2
import glob
import yaml
import torch
import random
import shutil
import pathlib
import numpy as np
import pandas as pd
from PIL import Image
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split

In [6]:
! wget https://storage.googleapis.com/air-lab-hackathon/Cocoa/cocoa_new.zip
! rm -rf sample_data
! unzip cocoa_new.zip
! rm cocoa_new.zip
! rm -rf __MACOSX

Streaming output truncated to the last 5000 lines.
  inflating: cocoa_new/valid/img_2510_JPG_jpg.rf.1b26ec4ea50a73ed84473a25058fe168.jpg  
  inflating: cocoa_new/valid/img_2954_JPG_jpg.rf.d86e11a6386434ec4c5e9bdbd0c66ecf.xml  
  inflating: cocoa_new/valid/img_3920_JPG_jpg.rf.7197873115df59abc4f4130485aba87a.jpg  
  inflating: cocoa_new/valid/img_2510_JPG_jpg.rf.1b26ec4ea50a73ed84473a25058fe168.xml  
  inflating: cocoa_new/valid/img_3082_JPG_jpg.rf.22386dfd0e7220f0261af55f754ac632.jpg  
  inflating: cocoa_new/valid/img_2988_JPG_jpg.rf.d5bd9411b7eba4016505af66b56b82f4.jpg  
  inflating: cocoa_new/valid/img_255_JPG_jpg.rf.3443a4428310daa987a86fdab5ca5e9a.xml  
  inflating: cocoa_new/valid/img_4084_JPG_jpg.rf.159bae477531188c72b3d3a96f66645c.jpg  
  inflating: cocoa_new/valid/img_885_JPG_jpg.rf.5cd9269fad3cbbc6fa995e0ee41b20a1.xml  
  inflating: cocoa_new/valid/img_885_JPG_jpg.rf.5cd9269fad3cbbc6fa995e0ee41b20a1.jpg  
  inflating: cocoa_new/valid/img_2988_JPG_jpg.rf.d5bd9411b7eba4016505af6

In [7]:
data_dir   = 'cocoa_new/train/'
data_dir   = pathlib.Path(data_dir).with_suffix('')
image_list = list(data_dir.glob('*.jpg'))
label_data = pd.read_csv('cocoa_new/train/label_map.csv')

def extract_bounding_boxes_norm(data_f, img_name):
  return data_f[data_f['Image id']==img_name][['xmin_norm', 'ymin_norm', 'xmax_norm', 'ymax_norm', 'Actual Label']].values

def extract_bounding_boxes(df, img_id):
  return df[df['Image id']==img_id][['xmin', 'ymin', 'xmax', 'ymax', 'Actual Label']].values

def generate_yolo_label(df, img_id, save_dir = 'yolo-data/base/labels/'):
  ext_boxes = extract_bounding_boxes_norm(df, img_id)
  img_width, img_height = (640,640)
  with open(save_dir+img_id[:-3]+'txt', 'w') as l_file:
    for b in ext_boxes:
      x1, y1, x2, y2, img_label =  b
      xc = x1+ (x2-x1)/2
      yc  = y1+ (y2-y1)/2
      l_file.write(f'{cocoa_label_map[img_label]} {x1:.6f} {y1:.6f} {xc:.6f} {yc:.6f}\n')

for d in ['base', 'train', 'val']:
  os.makedirs(f'yolo-data/{d}/images', exist_ok=True)
  os.makedirs(f'yolo-data/{d}/labels', exist_ok=True)
labels = label_data['Actual Label'].unique()
cocoa_label_map = {val:idx[0] for  idx, val in np.ndenumerate(labels)}

for i in image_list:
  generate_yolo_label(label_data, os.path.basename(i))

cocoa_label_map

{'Spoit': 0, 'Immature': 1, 'Mature_Unripe': 2, 'Ripped': 3}

In [8]:
base_labels = glob.glob('yolo-data/base/labels/*.txt')
print(f'Found {len(base_labels)} files.')
train, val = train_test_split(base_labels, test_size=0.1, random_state=42)
print(f'Train has {len(train)} samples')
print(f'Val has {len(val)} samples')

Found 2274 files.
Train has 2046 samples
Val has 228 samples


In [9]:
for f in train:
  i_name = os.path.basename(f)[:-3]+'jpg'
  shutil.copyfile(f, 'yolo-data/train/labels/'+os.path.basename(f))
  shutil.copyfile('cocoa_new/train/'+i_name, 'yolo-data/train/images/'+i_name)

for f in val:
  i_name = os.path.basename(f)[:-3]+'jpg'
  shutil.copyfile(f, 'yolo-data/val/labels/'+os.path.basename(f))
  shutil.copyfile('cocoa_new/train/'+i_name, 'yolo-data/val/images/'+i_name)

In [10]:
yaml_dict = {
    'path': '/content/yolo-data',
    'train': 'train/images',
    'val': 'val/images',
    'names': {idx[0]: val for  idx, val in np.ndenumerate(labels)}

}

with open('data_config.yaml', 'w+') as yml_file:
    yaml.dump(yaml_dict, yml_file)

In [12]:
batch = 96
epochs = 300
name = 'exp_v1.1'
optimizer = 'SGD'
project = 'cocoa-training'
model_version = 'yolov8n.pt'
data_config = '/content/data_config.yaml'

model = YOLO(model_version)
hist = model.train(
    imgsz=640,
    name=name,
    batch=batch,
    epochs=epochs,
    project=project,
    data=data_config,
    patience=0,
    optimizer=optimizer,
    dropout = 0.4,
    seed = 0,
    cos_lr = True,
    cache = True,
    plots=True
  )

Ultralytics YOLOv8.2.21 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data_config.yaml, epochs=300, time=None, patience=0, batch=96, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=cocoa-training, name=exp_v1.1, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 755k/755k [00:00<00:00, 19.4MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

    275/300      13.1G     0.6296     0.7341      1.193        373        640: 100%|██████████| 22/22 [00:12<00:00,  1.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.75it/s]

                   all        228       1219      0.722      0.602      0.684      0.506



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    276/300      12.9G     0.6385     0.7544      1.205        861        640:  64%|██████▎   | 14/22 [00:08<00:04,  1.67it/s]


KeyboardInterrupt: 

# Make Predictions

In [ ]:
test_data = glob.glob('/content/cocoa_new/test_new/*.jpg')
results = model.predict(
    test_data,
)
filenames = [os.path.basename(r.path) for idx, r in enumerate(results) for b in r.boxes.xyxyn.cpu().numpy()]
pred_boxes = np.concatenate([r.boxes.xyxyn.cpu().numpy() for r in results])
pred_classes = np.concatenate([r.boxes.cls.cpu().numpy() for r in results])
pred_confidence = np.concatenate([r.boxes.conf.cpu().numpy() for r in results])

In [ ]:
cocoa_label_map_rev = {cocoa_label_map[key]:key for key in cocoa_label_map}
pred_labels = list(map(lambda x : cocoa_label_map_rev[x], list(pred_classes.astype(int))))
cocoa_label_map_rev

In [ ]:
submission_df = pd.DataFrame(
    {
        'Image id': filenames,
        'Class': pred_labels,
        'Confidence': pred_confidence,
        'xmin': pred_boxes[:, 0],
        'ymin': pred_boxes[:, 1],
        'xmax': pred_boxes[:, 2],
        'ymax': pred_boxes[:, 3],
    }
)
submission_df.head()

In [ ]:
submission_df.to_csv('18446744073709551616.csv', index=False)